In [4]:
import os

from google.cloud import bigquery

In [5]:
os.environ["PROJECT_ID"] = "text-analysis-323506"

### Create a new Google Cloud service account to access the APIs programmatically.

__Don't run the following cell if you already have setup a cloud service account and have it's associated key__.

Following command creates a Google Cloud Service account, which is required to access APIs. This command should be run only once. When ran for the first time, it create a service account, details of which can be found in IAM & Admin section of GCP console.

After that, it creates a Google Cloud Service account key got the newly created account. Key is stored at /home/jupyter directory. Download this key. Upload it the next time when this notebook is to be run again. __Following cell should not be run more than once, unless we want to create a new cloud service account__.

In [ ]:
%%bash

# These should be run only once. These commands setup a cloud service account with required permissions to call APIs
gcloud iam service-accounts create my-api-sa --display-name "api account"
gcloud iam service-accounts keys create ~/key.json --iam-account my-api-sa@${PROJECT_ID}.iam.gserviceaccount.com

 #### Open cloud shell and run the command given below.
 This command gives the permission to read/write into bigquery to service account
 
```gcloud projects add-iam-policy-binding <GCP_PROJECT_ID> --member=serviceAccount:<GCP_Service_Account> --role=roles/bigquery.admin```

__Example__:

```gcloud projects add-iam-policy-binding text-analysis-323506 --member=serviceAccount:my-api-sa@text-analysis-323506.iam.gserviceaccount.com --role=roles/bigquery.admin```

Following command stores path to json file in an environment variable. This is required to access API endpoints using python.

In [6]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.abspath("/home/jupyter/key.json")

#### Create a Google Cloud bigquery api client object

In [7]:
bq_client = bigquery.Client()

### Important: 
#### Enable BigQuery API for this project in Google Cloud Platform console.

## Create dataset

In [54]:
dataset = bigquery.Dataset('text-analysis-323506.iris_dataset')

In [55]:
dataset.location = "us-east1"

In [56]:
dataset = bq_client.create_dataset(dataset, timeout=30)

In [57]:
print(f"Created dataset {bq_client.project}.{dataset.dataset_id}")

Created dataset text-analysis-323506.iris_dataset


#### New dataset can be seen at bigquery UI

## Create Table

BigQuery database is a managed SQL database. So before creating the table, we need to mention schema, just like how it's done in case od a plain SQL database

In [58]:
schema = [
        bigquery.SchemaField("Id", "INTEGER", mode="REQUIRED"),
        bigquery.SchemaField("SepalLengthCm", "FLOAT", mode="REQUIRED"),
        bigquery.SchemaField("SepalWidthCm", "FLOAT", mode="REQUIRED"),
        bigquery.SchemaField("PetalLengthCm", "FLOAT", mode="REQUIRED"),
        bigquery.SchemaField("PetalWidthCm", "FLOAT", mode="REQUIRED"),
        bigquery.SchemaField("Species", "STRING", mode="REQUIRED")
    ]

In [59]:
table_id = 'text-analysis-323506.iris_dataset.iris_table'

In [60]:
table = bigquery.Table(table_id, schema=schema)

In [61]:
table = bq_client.create_table(table)

In [62]:
print(f"Created table {table.project}.{table.dataset_id}.{table.table_id}")

Created table text-analysis-323506.iris_dataset.iris_table


## Load Data into BigQuery Table

Csv files need to be in cloud storage for loading to work.

In [63]:
! gsutil -m cp ./data/Iris.csv gs://text-analysis-323506/

Copying file://./data/Iris.csv [Content-Type=text/csv]...
/ [1/1 files][  5.0 KiB/  5.0 KiB] 100% Done                                    
Operation completed over 1 objects/5.0 KiB.                                      


In [64]:
job_config = bigquery.LoadJobConfig(
    schema=schema,
    skip_leading_rows=1,
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
    # WRITE_TRUNCATE replaces existing data 
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND
)

In [65]:
uri = "gs://text-analysis-323506/Iris.csv"

load_job = bq_client.load_table_from_uri(
    uri, table_id, job_config=job_config, 
)

load_job.result()  # Waits for the job to complete.

LoadJob<project=text-analysis-323506, location=us-east1, id=0afc3092-3a91-4f9f-83c6-ad575396a332>

In [66]:
destination_table = bq_client.get_table(table_id)
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 150 rows.


#### BigQuery UI shows all 150 rows loaded into the table

## Run SQL Queries on data stored in the table

A simple SQL query in the following cell finds the count of flowers in our dataset which have sepal length values between 5 cm and 6 cm.

In [105]:
query = """
        SELECT count(Species)
        FROM `text-analysis-323506.iris_dataset.iris_table`
        WHERE SepalLengthCm>5.0 AND SepalLengthCm<6.0
    """

In [106]:
# Make an API request
results = bq_client.query(query)

In [107]:
# Proper way to access the rows in results

# for row in results:
#     print(row)

In [109]:
print(f"Number of flowers which have sepal length between 5.0 cm and 6.0 cm in our dataset: {list(results)[0][0]}")

Number of flowers which have sepal length between 5.0 cm and 6.0 cm in our dataset: 51


#### Any query can be run the same way above. No matter how complex it maybe is. API fetches us the results of the query.

## Delete Tables

In [48]:
table_id = 'text-analysis-323506.iris_dataset.iris_table'

In [50]:
bq_client.delete_table(table_id, not_found_ok=True)

#### BigQuery UI shows that the table has been deleted

## Delete Databases

In [51]:
dataset_id = 'text-analysis-323506.iris_dataset'

In [53]:
bq_client.delete_dataset(dataset_id, delete_contents=True, not_found_ok=True)

#### BigQuery UI shows that the dataset has been deleted